In [1]:
from src.model.shap.run_shap import run_shap

shap_explanations = run_shap(
    dataset_name="demographic+digital",
    target_col="group",
    task="classification",
    model_name_to_explain="SVC",
    timestamp="2025-09-12_1559"
)

/home/gianluca/Research/tmt-analysis/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers._permutation.PermutationExplainer | link: CPUDispatcher
[SHAP] backend: shap.explainers.

In [2]:
shap_explanations[0].values.shape

(1, 20, 2)

In [7]:
import shap


def extract_positive_class_explanations(explanations):
    positive_class_explanations = []
    for explanation in explanations:
        vals = explanation.values.copy()
        base = explanation.base_values.copy()
        positive_class_idx = 1
        # Caso 1: (n_samples, n_classes, n_features)
        if vals.ndim == 3 and vals.shape[1] == 2:
            vals = vals[:, positive_class_idx, :]
            base = base[:, positive_class_idx] if base.ndim == 2 else base

        # Caso 2: (n_samples, n_features, n_classes)
        elif vals.ndim == 3 and vals.shape[2] == 2:
            vals = vals[:, :, positive_class_idx]
            base = base[:, positive_class_idx] if base.ndim == 2 else base

        positive_class_explanation = shap.Explanation(
            values=vals,
            base_values=base,
            data=explanation.data,
            feature_names=explanation.feature_names
        )

        positive_class_explanations.append(positive_class_explanation)

        return positive_class_explanations


In [8]:
positive_class_explanations = extract_positive_class_explanations(shap_explanations)
positive_class_explanations

[.values =
 array([[-0.02841402,  0.00256862, -0.00823842, -0.00495456, -0.00809279,
         -0.0104945 ,  0.04862322, -0.00230373,  0.0208153 , -0.00460962,
          0.0031431 ,  0.05347718, -0.00239226, -0.006643  , -0.00307571,
          0.06770231, -0.01478016, -0.01080193,  0.02321736,  0.00434233]])
 
 .base_values =
 array([0.51942199])
 
 .data =
 array([[ 1.22870466,  0.49057918, -0.83894   , -0.0395364 ,  0.14298255,
          0.62999382, -0.69400903,  0.62892401,  0.70820402,  0.0282486 ,
          0.50892069,  1.80972832,  0.51196432, -1.0600834 , -0.59419224,
         -1.22383699,  0.5049662 , -1.12254671, -0.34409597,  0.45018772]])]

In [11]:

import pandas as pd


def build_shap_absolute_df(explanations):
    # Lista de dicts: {feature: abs(shap)}
    rows = []

    for expl in explanations:
        absolute_values = np.abs(expl.values)  # (1, n_features)
        names = expl.feature_names
        # Armo dict con valores de este fold
        row = dict(zip(names, absolute_values.flatten()))
        rows.append(row)

    # DataFrame folds × features (NaN si el feature no apareció en ese fold)
    df = pd.DataFrame(rows)
    return df

In [12]:
df = build_shap_absolute_df(positive_class_explanations)
df


,average_duration_PART_B,distance_difference_from_ideal_PART_A,hesitation_time_PART_A,intra_target_time_PART_A,max_duration_PART_B,non_cut_correct_targets_touches_PART_A,non_cut_correct_targets_touches_PART_B,non_cut_rt_PART_A,non_cut_rt_PART_B,rt_PART_A,rt_PART_B,search_time_PART_B,total_distance_PART_A,travel_distance_PART_B,travel_time_PART_A,non_cut_correct_targets_touches_B_A_ratio,state_transitions_B_A_ratio,travel_avg_speed_B_A_ratio,is_valid_sum_B,age
0,0.028414,0.002569,0.008238,0.004955,0.008093,0.010495,0.048623,0.002304,0.020815,0.00461,0.003143,0.053477,0.002392,0.006643,0.003076,0.067702,0.01478,0.010802,0.023217,0.004342


In [14]:
# Frecuencia de aparición (proporción de folds donde no es NaN)
freq = df.notna().mean()

# Promedio condicional (solo folds con valor, ignora NaN)
mean_cond = df.mean(skipna=True)

# Resultado combinado
result = pd.DataFrame({
    "mean_abs_shap_cond": mean_cond,
    "freq_selection": freq
}).sort_values("mean_abs_shap_cond", ascending=False)

result

,mean_abs_shap_cond,freq_selection
non_cut_correct_targets_touches_B_A_ratio,0.067702,1.0
search_time_PART_B,0.053477,1.0
non_cut_correct_targets_touches_PART_B,0.048623,1.0
average_duration_PART_B,0.028414,1.0
is_valid_sum_B,0.023217,1.0
non_cut_rt_PART_B,0.020815,1.0
state_transitions_B_A_ratio,0.014780,1.0
travel_avg_speed_B_A_ratio,0.010802,1.0
non_cut_correct_targets_touches_PART_A,0.010495,1.0
hesitation_time_PART_A,0.008238,1.0
